In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2024-09-21 17:09:45.268406: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Num GPUs Available:  1


In [14]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Reshape, Activation, Dropout, GlobalAveragePooling1D, TimeDistributed

class VisionTransformer(tf.keras.layers.Layer):
    def __init__(self, num_heads, embed_dim, num_transformer_blocks, patch_size, num_patches):
        super(VisionTransformer, self).__init__()
        self.num_heads = num_heads
        self.embed_dim = embed_dim
        self.num_transformer_blocks = num_transformer_blocks
        self.patch_size = patch_size
        self.num_patches = num_patches

        self.patch_dim = patch_size * patch_size

        self.patch_embedding = layers.Dense(embed_dim)  # Linear projection for patches
        self.pos_embedding = tf.Variable(tf.random.normal([1, num_patches, embed_dim]))  # Positional encoding
        self.transformer_blocks = [
            tf.keras.layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embed_dim)
            for _ in range(self.num_transformer_blocks)
        ]

    def call(self, x):
        batch_size = tf.shape(x)[0]
        patches = tf.image.extract_patches(
            images=x,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding='VALID'
        )
        patch_dims = self.patch_size * self.patch_size
        x = tf.reshape(patches, [batch_size, self.num_patches, patch_dims])
        x = self.patch_embedding(x) + self.pos_embedding

        for transformer in self.transformer_blocks:
            x = transformer(x, x)
        return x

input_1 = Input(shape=(None, 25, 25, 1))
input_2 = Input(shape=(None, 25, 25, 1))
input_3 = Input(shape=(None, 25, 25, 1))

patch_size = 5
num_patches = (25 // patch_size) ** 2

vit_layer = VisionTransformer(num_heads=8, embed_dim=64, num_transformer_blocks=4, patch_size=patch_size, num_patches=num_patches)

vit_output_1 = TimeDistributed(vit_layer)(input_1)
vit_output_2 = TimeDistributed(vit_layer)(input_2)
vit_output_3 = TimeDistributed(vit_layer)(input_3)

pooled_output_1 = tf.reduce_mean(vit_output_1, axis=2)
pooled_output_2 = tf.reduce_mean(vit_output_2, axis=2)
pooled_output_3 = tf.reduce_mean(vit_output_3, axis=2)

merged = tf.concat([pooled_output_1, pooled_output_2, pooled_output_3], axis=-1)

reshaped_merged = Reshape((-1, 64*3))(merged) 
lstm_out = LSTM(units=256, return_sequences=True, recurrent_activation='sigmoid')(reshaped_merged)
lstm_out = LSTM(units=256, return_sequences=True, recurrent_activation='sigmoid')(lstm_out)
lstm_out = Dropout(0.2)(lstm_out)

dense_out = Dense(units=100)(lstm_out)
dense_out = Dense(units=625)(dense_out)

dense_out = Dropout(0.2)(dense_out)

output = Activation('softmax')(dense_out)
model = Model(inputs=[input_1, input_2, input_3], outputs=output)

def cancel_loss(y_true, y_pred):
    y_true_truncated = y_true[:, 40:, :, :]
    y_pred_truncated = y_pred[:, 40:, :, :]
    loss = tf.keras.losses.categorical_crossentropy(y_true_truncated, y_pred_truncated)
    return tf.reduce_mean(loss)

model.compile(optimizer='adam', loss=cancel_loss)
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_38 (InputLayer)          [(None, None, 25, 2  0           []                               
                                5, 1)]                                                            
                                                                                                  
 input_39 (InputLayer)          [(None, None, 25, 2  0           []                               
                                5, 1)]                                                            
                                                                                                  
 input_40 (InputLayer)          [(None, None, 25, 2  0           []                               
                                5, 1)]                                                        

In [3]:
!pip install scikit-learn

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [3]:
import os
import numpy as np
import tensorflow as tf

def data_generator(file_list, data_dir, batch_size):
    total_files = len(file_list)
    
    while True:
        np.random.shuffle(file_list)
        for i in range(0, total_files, batch_size):
            batch_files = file_list[i:i + batch_size]
            feat_batch, army_batch, y_batch = [], [], []

            for file in batch_files:
                file_path = os.path.join(data_dir, file)
                
                data = np.load(file_path, allow_pickle=True)
                
                feat_batch.append(data['feat_padded'].astype(np.float32))
                army_batch.append(data['army_padded'].astype(np.float32))
                Y_padded = data['Y_padded'].astype(np.float32)
                
                # print(np.max(Y_padded[:,0]),np.max(Y_padded[:,1]))
                y_dim1_one_hot = tf.keras.utils.to_categorical(Y_padded[:, 0], num_classes=25)
                y_dim2_one_hot = tf.keras.utils.to_categorical(Y_padded[:, 1], num_classes=25)
                y_one_hot = np.stack([y_dim1_one_hot, y_dim2_one_hot], axis=1)
                y_batch.append(y_one_hot.astype(np.float32))
            
            feat_batch = np.array(feat_batch)
            army_batch = np.array(army_batch)
            y_batch = np.array(y_batch)
            # print(feat_batch.shape)
            yield (feat_batch, army_batch), y_batch


In [10]:
import os
from sklearn.model_selection import train_test_split

data_dir = "autodl-tmp/Numpy_Data"
file_names = os.listdir(data_dir)

train_files, test_files = train_test_split(file_names, test_size=0.1, random_state=42)  # 80% train, 20% test
train_files, val_files = train_test_split(train_files, test_size=0.1, random_state=42)  # 75% train, 25% val

In [11]:
print(len(train_files),len(val_files),len(test_files))

2345 261 290


In [7]:
file_path = '/root/autodl-tmp/Numpy_Data/data_row_16.npz'
data = np.load(file_path, allow_pickle=True)
data_keys = data.files
print(data_keys)
# print(data['Y_padded'].shape)
Y_padded = data['Y_padded']
print(Y_padded.shape)
print("Max value in Y_padded[:, 1]:", np.max(Y_padded[:, 0]))

['feat_padded', 'army_padded', 'Y_padded']
(1000, 2)
Max value in Y_padded[:, 1]: 5


In [12]:
batch_size = 16

train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(train_files, data_dir, batch_size),
    output_signature=(
        (tf.TensorSpec(shape=(None, None, 25, 25), dtype=tf.float32),  # feat_padded
         tf.TensorSpec(shape=(None, None, 25, 25), dtype=tf.float32)),  # army_padded
        tf.TensorSpec(shape=(None, None, 2, 25), dtype=tf.float32)  # Y_padded
    )
)
train_dataset = train_dataset.shuffle(buffer_size=100)

val_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(val_files, data_dir, batch_size),
    output_signature=(
        (tf.TensorSpec(shape=(None, None, 25, 25), dtype=tf.float32),  # feat_padded
         tf.TensorSpec(shape=(None, None, 25, 25), dtype=tf.float32)),  # army_padded
        tf.TensorSpec(shape=(None, None, 2, 25), dtype=tf.float32)  # Y_padded
    )
)
# val_dataset = val_dataset.batch(batch_size)

test_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(test_files, data_dir, batch_size),
    output_signature=(
        (tf.TensorSpec(shape=(None, None, 25, 25), dtype=tf.float32),  # feat_padded
         tf.TensorSpec(shape=(None, None, 25, 25), dtype=tf.float32)),  # army_padded
        tf.TensorSpec(shape=(None, None, 2, 25), dtype=tf.float32)  # Y_padded
    )
)
# test_dataset = test_dataset.batch(batch_size)

In [9]:
# for batch in train_dataset.take(1):  # Inspect 1 batch
#     (feat_padded, army_padded), y_padded = batch
#     print("feat_padded shape:", feat_padded.shape)
#     print("army_padded shape:", army_padded.shape)
#     print("y_padded shape:", y_padded.shape)

2024-09-21 12:03:42.975300: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 23 of 100
2024-09-21 12:03:52.970563: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 48 of 100
2024-09-21 12:04:02.810704: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 71 of 100
2024-09-21 12:04:12.856307: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 95 of 100


feat_padded shape: (16, 1000, 25, 25)
army_padded shape: (16, 1000, 25, 25)
y_padded shape: (16, 1000, 2, 25)


2024-09-21 12:04:14.755920: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.


In [10]:
print(train_dataset)

<ShuffleDataset element_spec=((TensorSpec(shape=(None, None, 25, 25), dtype=tf.float32, name=None), TensorSpec(shape=(None, None, 25, 25), dtype=tf.float32, name=None)), TensorSpec(shape=(None, None, 2, 25), dtype=tf.float32, name=None))>


In [13]:
from tensorflow.keras.callbacks import EarlyStopping

total_files = len(os.listdir("autodl-tmp/Numpy_Data"))
train_steps = len(train_files) // batch_size
validation_steps = len(val_files) // batch_size

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(train_dataset, 
          epochs=100, 
          steps_per_epoch=train_steps, 
          validation_data=val_dataset,
          validation_steps=validation_steps,
          callbacks=[early_stopping])

model.save('model_full.h5')

Epoch 1/100


2024-09-21 17:13:01.283258: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 49 of 100
2024-09-21 17:13:11.215276: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 98 of 100


  1/146 [..............................] - ETA: 49:42 - loss: 1.3527

2024-09-21 17:13:11.616350: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.


146/146 [==============================] - 68s 324ms/step - loss: 1.0509 - val_loss: 0.7465
Epoch 2/100
146/146 [==============================] - 45s 312ms/step - loss: 0.9968 - val_loss: 0.7326
Epoch 3/100
146/146 [==============================] - 46s 316ms/step - loss: 0.9726 - val_loss: 0.7538
Epoch 4/100
146/146 [==============================] - 46s 317ms/step - loss: 0.9315 - val_loss: 0.7708
Epoch 5/100
146/146 [==============================] - 45s 311ms/step - loss: 0.8759 - val_loss: 0.7785
Epoch 6/100
146/146 [==============================] - 46s 312ms/step - loss: 0.8394 - val_loss: 0.8292
Epoch 7/100
146/146 [==============================] - 45s 311ms/step - loss: 0.7618 - val_loss: 0.8989
Epoch 8/100
146/146 [==============================] - 47s 322ms/step - loss: 0.7146 - val_loss: 0.8925
Epoch 9/100
146/146 [==============================] - 47s 324ms/step - loss: 0.6475 - val_loss: 0.9582
Epoch 10/100
146/146 [==============================] - 48s 326ms/step - los

In [16]:
test_steps = len(test_files) // batch_size
model.evaluate(test_dataset, steps=test_steps)

12/18 [===================>..........] - ETA: 1s - loss: 0.7503

2024-09-21 17:23:19.110945: W tensorflow/core/framework/op_kernel.cc:1733] UNKNOWN: FileNotFoundError: [Errno 2] No such file or directory: 'autodl-tmp/Numpy_Data/.data_row_3603.npz.zbLyAn'
Traceback (most recent call last):

  File "/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1030, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/tmp/ipykernel_1228/1175695857.py", line 17, in data_generator
    data = np.load(file_path, allow_pickle=True)

  File "/root/miniconda3/lib/python3.8/site-packages/numpy/lib/npyio.py", line 390, in load
    fid = stack.enter_context(open(os_fspath(file), "rb"))

FileNotFoundError: [Errn

UnknownError: Graph execution error:

2 root error(s) found.
  (0) UNKNOWN:  FileNotFoundError: [Errno 2] No such file or directory: 'autodl-tmp/Numpy_Data/.data_row_3603.npz.zbLyAn'
Traceback (most recent call last):

  File "/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1030, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/tmp/ipykernel_1228/1175695857.py", line 17, in data_generator
    data = np.load(file_path, allow_pickle=True)

  File "/root/miniconda3/lib/python3.8/site-packages/numpy/lib/npyio.py", line 390, in load
    fid = stack.enter_context(open(os_fspath(file), "rb"))

FileNotFoundError: [Errno 2] No

In [13]:
from sklearn.model_selection import train_test_split

X1_train, X1_test, X2_train, X2_test, y_train, y_test = train_test_split(
    feat_padded, army_padded, Y_padded, test_size=0.2, random_state=42
)

ValueError: Found input variables with inconsistent numbers of samples: [16, 16, 1000]

In [ ]:
del feat_padded, army_padded, Y_padded
gc.collect()

In [ ]:
y_train = np.squeeze(y_train, axis=2)
y_test = np.squeeze(y_test, axis=2)

In [ ]:
num_classes = 25

y_train_class_1 = tf.one_hot(y_train[..., 0], depth=num_classes)
y_train_class_2 = tf.one_hot(y_train[..., 1], depth=num_classes)
y_train_categorical = tf.stack([y_train_class_1, y_train_class_2], axis=-2)

y_test_class_1 = tf.one_hot(y_test[..., 0], depth=num_classes)
y_test_class_2 = tf.one_hot(y_test[..., 1], depth=num_classes)
y_test_categorical = tf.stack([y_test_class_1, y_test_class_2], axis=-2)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.fit([X1_train, X2_train], y_train_categorical, epochs=100, batch_size=16, verbose=1)
model.save('model_full.h5')
model.evaluate([X1_test, X2_test], y_test_categorical, verbose=1)